In [ ]:
import pandas as pd
import os
import matplotlib.pylab as plt
import xarray as xr
import numpy as np
from scipy.signal import butter, filtfilt
from matplotlib.ticker import ScalarFormatter
from matplotlib.dates import DateFormatter
from glob import glob
import gsw
import cartopy.crs as ccrs
from matplotlib.colors import LightSource
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from matplotlib.colors import LightSource, LinearSegmentedColormap, Normalize
from matplotlib.cm import ScalarMappable

import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec


In [ ]:
# BPR is at a depth of 2321 m, original data has a sample rate of 1 second

file_path = '/Users/sandraslead/URI/Research/Chapter3/Endeavor/ONC_Endeavor_BPR/'
file_list = ['NRCANBPR80_SeafloorPressure_20230301T160243Z_20230313T054923Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230313T054924Z_20230324T193603Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230324T193604Z_20230405T092244Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230405T092245Z_20230416T230924Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230416T230925Z_20230428T125605Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230428T125606Z_20230510T024245Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230510T024246Z_20230521T162926Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230521T162927Z_20230602T061606Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230602T061607Z_20230613T200247Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230613T200248Z_20230625T094927Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230625T094928Z_20230706T233607Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230706T233608Z_20230718T132246Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230718T132247Z_20230730T030926Z-NaN_clean.csv',
             'NRCANBPR80_SeafloorPressure_20230730T030927Z_20230730T160242Z-NaN_clean.csv',]

file_list.sort()

def read_and_clean_csv(filepath, actual_data_line):
    try:
        df = pd.read_csv(filepath, skiprows=actual_data_line, names=['Date', 'bottom_pressure', 'QC Flag'],\
                         header=None, parse_dates=['Date'],low_memory=False)
        df['bottom_pressure'] = pd.to_numeric(df['bottom_pressure'], errors='coerce')
        df = df.dropna(subset=['bottom_pressure'])
        df.set_index('Date', inplace=True)
        hourly_df = df.resample('H').mean(numeric_only=True)
        return hourly_df
    except Exception as e:
        print(f"Error processing file {os.path.basename(filepath)}: {e}")
        return pd.DataFrame()

concatenated_dfs = [read_and_clean_csv(os.path.join(file_path, file), 53) for file in file_list]

bpr_df = pd.concat([df for df in concatenated_dfs if not df.empty])

bpr_df.reset_index(inplace=True)

# Plotting the pressure
plt.figure(figsize=(12, 6))
plt.plot(bpr_df['Date'], bpr_df['bottom_pressure'])
plt.title('Bottom Pressure')
plt.ylabel('Bottom Pressure (dbar)')
plt.grid(True)
plt.show()

## Detide

In [ ]:
bpr_df = bpr_df.dropna()

# Sampling frequency and Nyquist frequency in hours^-1
sampling_rate_hours = 1  
nyquist_frequency = 0.5 * sampling_rate_hours

# Define the cutoff frequency for 48 hours
cutoff_frequency_hours = 1 / 48

# Normalize the cutoff frequency with respect to the Nyquist frequency
normalized_cutoff_frequency = cutoff_frequency_hours / nyquist_frequency

# butterworth low pass filter
N = 6
b, a = butter(N, normalized_cutoff_frequency, btype='low', analog=False)
bpr_df['detided_bottom_pressure'] = filtfilt(b, a, bpr_df['bottom_pressure'])

# The 50 first rows removed due to filter initialization effects.
detided_bpr_df = bpr_df.iloc[50:]

plt.figure(figsize=(12, 6))
plt.plot(detided_bpr_df['Date'], detided_bpr_df['detided_bottom_pressure'])
plt.title('Detided Bottom Pressure')
plt.ylabel('Bottom Pressure (dbar)')
plt.grid(True);

## Detrend

In [ ]:
x_values = np.arange(len(detided_bpr_df))

poly_degree = 3
coefs = np.polyfit(x_values, detided_bpr_df['detided_bottom_pressure'], poly_degree)
bpr_trend = np.polyval(coefs, x_values)

detided_bpr_df = detided_bpr_df.copy()
detided_bpr_df.loc[:, 'detrended_bottom_pressure'] = detided_bpr_df['detided_bottom_pressure'] - bpr_trend

plt.figure(figsize=(12, 6))
plt.plot(detided_bpr_df.index, detided_bpr_df['detided_bottom_pressure'], label='Original')
plt.plot(detided_bpr_df.index, bpr_trend, label='Trend', linestyle='--')
plt.legend()


plt.figure(figsize=(12, 6))
plt.plot(detided_bpr_df.index, detided_bpr_df['detrended_bottom_pressure'], label='Detrended', alpha=0.7)
plt.legend();


# Read in SWOT Data

In [ ]:
def find_closest_ssh_point(file_path, target_lat, target_lon):
    ds = xr.open_dataset(file_path)
    target_lon = target_lon % 360
    
    lat_diff = abs(ds.latitude - target_lat)
    lon_diff = abs(ds.longitude - target_lon)
    total_diff = lat_diff + lon_diff
    min_diff_idx = np.unravel_index(total_diff.argmin(), total_diff.shape)
    
    # Extract the swot value and time at the closest location
    closest_ssh = ds.ssha.isel(num_lines=min_diff_idx[0], num_pixels=min_diff_idx[1]).values.item()
    closest_time = pd.to_datetime(ds.time.isel(num_lines=min_diff_idx[0]).values)
    
    return closest_time, closest_ssh

def process_swot_files(folder_path, target_lat, target_lon):
    swot_values = []
    for file_name in sorted(os.listdir(folder_path)):
        if file_name.endswith('.nc'):
            file_path = os.path.join(folder_path, file_name)
            time, ssh = find_closest_ssh_point(file_path, target_lat, target_lon)
            swot_values.append({'Date': time, 'ssh': ssh})
            
    return pd.DataFrame(swot_values)

folder_path = '/Users/sandraslead/URI/Research/Chapter3/Endeavor/SWOT_data/Pass026'
target_lat = 47.948583
target_lon = -129.098678  
swot_time_series = process_swot_files(folder_path, target_lat, target_lon)

swot_time_series['Date'] = pd.to_datetime(swot_time_series['Date'], errors='coerce', utc=True)

plt.figure(figsize=(12, 6))
plt.plot(swot_time_series['Date'], swot_time_series['ssh'])
plt.title('SWOT SSH data')
plt.ylabel('ssh (m)')
plt.grid(True)

# Read in Aviso Data

In [ ]:
columns = ['Julian Day', 'SSH cm', 'Year', 'Month', 'Day', 'Time']
aviso = pd.read_csv('Aviso_SSH/aviso_endeavorsite.txt', sep='\s+', names=columns)

aviso['Year'] = aviso['Year'].astype(str)
aviso['Month'] = aviso['Month'].astype(str)
aviso['Day'] = aviso['Day'].astype(str)

aviso['Day'] = aviso['Day'].str.zfill(2)

aviso['Date'] = pd.to_datetime(aviso['Year'] + '-' + aviso['Month'] + '-' + aviso['Day'], 
                               format='%Y-%b-%d', errors='coerce', utc=True)

aviso['SSH m'] = aviso['SSH cm'] / 100

#aviso.drop(columns=['Julian Day', 'SSH cm', 'Year', 'Month', 'Day', 'Date_Str'], inplace=True)

aviso = aviso[13:-30].reset_index(drop=True)

# Calculate seafloor pressure from SSH

In [ ]:
def calc_sfp_ssh(df, ssh_col_name, g=9.81):
    pressure_series = df[ssh_col_name] * g * 1030 / 10000
    return pressure_series.values

# For SWOT ssh
swot_time_series['pressure'] = calc_sfp_ssh(swot_time_series, 'ssh')
depth = 2195
depth_dbar = depth * 9.81 * 1030 / 10000  # Convert depth to pressure in dbar
swot_time_series['pressure'] += depth_dbar

# For Aviso ssh
aviso['pressure'] = calc_sfp_ssh(aviso, 'SSH m') + depth_dbar

# Compare seafloor pressure from BPR, SWOT, and Aviso

In [ ]:
def calculate_anomaly(df, column_name):
    return df[column_name] - np.mean(df[column_name])

detided_bpr_df['pressure_anomaly'] = calculate_anomaly(detided_bpr_df, 'detided_bottom_pressure')
swot_time_series['pressure_anomaly'] = calculate_anomaly(swot_time_series, 'pressure')
aviso['pressure_anomaly'] = calculate_anomaly(aviso, 'pressure')

if not detided_bpr_df.index.name or 'Date' not in detided_bpr_df.index.name.lower():
    if 'Date' in detided_bpr_df.columns:
        detided_bpr_df.set_index('Date', inplace=True)
    else:
        pass

detided_bpr_daily = detided_bpr_df.resample('D').mean()

if 'Date' not in swot_time_series.columns or not pd.api.types.is_datetime64_any_dtype(swot_time_series['Date']):
    pass
else:
    swot_time_series.set_index('Date', inplace=True)

swot_daily = swot_time_series.resample('D').mean() 
merged_data = pd.merge(swot_daily, detided_bpr_daily, 
                       left_index=True, right_index=True, 
                       how='inner', 
                       suffixes=('_swot', '_bpr'))

merged_data = pd.merge(swot_daily, detided_bpr_daily, 
                       left_index=True, right_index=True, 
                       how='inner', 
                       suffixes=('_swot', '_bpr'))

correlation = merged_data['pressure_anomaly_swot'].corr(merged_data['pressure_anomaly_bpr'])

# Plotting
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(detided_bpr_daily.index, detided_bpr_daily['pressure_anomaly'], label='BPR Data', color='C0')
ax.scatter(swot_time_series.index, swot_time_series['pressure_anomaly'], label='Bottom Pressure from SWOT SSH', color='C1')
ax.scatter(aviso['Date'], aviso['pressure_anomaly'], label='Bottom Pressure from Aviso SSH', color='C2')

# Display correlation
ax.text(0.7, 0.1, f'r (bpr and swot) = {correlation:.2f}', transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'))
plt.legend()
plt.ylabel('Pressure Anomaly (dbar)')
plt.grid(True)

# Save the figure
#plt.savefig('/mnt/data/SSH_vs_BPR_scatter.png', dpi=300, bbox_inches='tight', transparent=False)

# Remove Aviso trend from SWOT

In [ ]:
if not pd.api.types.is_datetime64_any_dtype(aviso['Date']):
    aviso['Date'] = pd.to_datetime(aviso['Date'])

aviso['days_since_start'] = (aviso['Date'] - aviso['Date'].min()).dt.days

if not isinstance(swot_time_series.index, pd.DatetimeIndex):
    swot_time_series.set_index(pd.to_datetime(swot_time_series['Date']), inplace=True)

swot_time_series['days_since_start'] = (swot_time_series.index - aviso['Date'].min()).days

poly_degree = 10  
aviso_coefs = np.polyfit(aviso['days_since_start'], aviso['pressure'], poly_degree)

swot_trend = np.polyval(aviso_coefs, swot_time_series['days_since_start'])

swot_time_series['detrended_pressure'] = swot_time_series['pressure'] - swot_trend



if 'Date' in detided_bpr_df.columns and not pd.api.types.is_datetime64_any_dtype(detided_bpr_df['Date']):
    detided_bpr_df['Date'] = pd.to_datetime(detided_bpr_df['Date'])
    detided_bpr_df.set_index('Date', inplace=True)
detided_bpr_daily = detided_bpr_df.resample('D').mean()

swot_time_series['detrended_pressure_anomaly'] = calculate_anomaly(swot_time_series, 'detrended_pressure')

swot_daily = swot_time_series.resample('D').mean()

correlation = swot_daily['detrended_pressure_anomaly'].corr(detided_bpr_daily['pressure_anomaly'])

# Plotting, including the correlation
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(detided_bpr_df.index, detided_bpr_df['pressure_anomaly'], label='BPR Data', color='C0')
ax.scatter(aviso['Date'], aviso['pressure_anomaly'], label='Bottom Pressure from Aviso SSH', color='C2')
ax.scatter(swot_time_series.index, swot_time_series['detrended_pressure_anomaly'], label='SWOT Pressure Detrended Using Aviso', color='C1')

# Add correlation text
ax.text(0.05, 0.95, f'r (detrended swot vs. bpr) = {correlation:.2f}', transform=ax.transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", edgecolor='black', facecolor='white'))

plt.legend(loc='lower right')
plt.ylabel('Pressure Anomaly (dbar)')
plt.grid(True)

# Save the figure
#plt.savefig('Aviso_trend_scatter.png', dpi=300, bbox_inches='tight', transparent=False)


In [ ]:
import matplotlib.dates as mdates

date_range = pd.date_range(start=swot_time_series.index.min(), end=swot_time_series.index.max(), freq='D')
days_since_start_range = (date_range - aviso['Date'].min()).days

trend_values = np.polyval(aviso_coefs, days_since_start_range)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(detided_bpr_df.index, detided_bpr_df['pressure_anomaly'], label='BPR Data', color='C0')
ax.scatter(aviso['Date'], aviso['pressure_anomaly'], label='Bottom Pressure from Aviso SSH', color='C2')
ax.scatter(swot_time_series.index, swot_time_series['detrended_pressure_anomaly'], label='SWOT Pressure Detrended Using Aviso', color='C1')
ax.scatter(swot_time_series.index, swot_time_series['pressure_anomaly'], label='Bottom pressure from SWOT SSH', color='C4')

ax.plot(date_range, trend_values - np.mean(trend_values)-0.012, label='Trend from Aviso', color='C3', linestyle='--')

plt.legend(loc = 'upper left')
plt.ylabel('Pressure Anomaly (dbar)')
plt.grid(True)

plt.savefig('Aviso_trend_scatter_with_trend.png', dpi=300, bbox_inches='tight', transparent=False)

# DAC (Dynamic Atmosphere Correction) Data

In [ ]:
file_path = 'DynamicAtmosphereCorrection/dac_endeavor.txt'
dac = pd.read_csv(file_path, delim_whitespace=True, header=None, names=['Julian Day', 'ssh_correction', 'Year', 'Month', 'Day', 'Time'])
dac['Datetime'] = dac['Year'].astype(str) + ' ' + dac['Month'] + ' ' + dac['Day'].astype(str) + ' ' + dac['Time']
dac['Datetime'] = pd.to_datetime(dac['Datetime'], format='%Y %b %d %H:%M')
dac['Datetime'] = dac['Datetime'].dt.tz_localize('UTC')
dac['ssh_correction_m'] = dac['ssh_correction'] / 100

fig, ax1 = plt.subplots(figsize=(12, 4))

ax1.plot(detided_bpr_df.index, detided_bpr_df['pressure_anomaly'], label='BPR', color='C0')
ax1.set_ylabel('Pressure anomaly (dbar)', color='C0') 
ax1.tick_params(axis='y', labelcolor='C0') 

ax2 = ax1.twinx()  
ax2.plot(dac['Datetime'], dac['ssh_correction_m'], label='DAC', color='C1')
ax2.set_ylabel('SSH Correction (m)', color='C1')  
ax2.tick_params(axis='y', labelcolor='C1') 

handles, labels = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()

combined_handles = handles + handles2
combined_labels = labels + labels2

ax1.legend(combined_handles, combined_labels, loc='best')

ax1.grid(True)
plt.savefig('endeavor_BPR_DAC.png', dpi=300, bbox_inches='tight', transparency = False)

# Add DAC back in to SWOT and re-calculate seafloor pressure

In [ ]:
dac.sort_values(by='Datetime', inplace=True)

merged_df = pd.merge_asof(
    swot_time_series,
    dac[['Datetime', 'ssh_correction_m']],
    left_on='Date',
    right_on='Datetime',
    direction='nearest'  
)

merged_df['ssh_with_dac'] = merged_df['ssh'] + merged_df['ssh_correction_m'].fillna(0)
swot_time_series['ssh_with_dac'] = merged_df['ssh_with_dac'].values

swot_time_series['pressure_with_DAC'] = calc_sfp_ssh(swot_time_series, 'ssh_with_dac')
depth = 2195
depth_dbar = depth * 9.81 * 1030 / 10000  
swot_time_series['pressure_with_DAC'] += depth_dbar


fig, ax1 = plt.subplots(figsize=(12, 4))

ax1.plot(detided_bpr_df.index, detided_bpr_df['pressure_anomaly'], label='BPR', color='C0')
ax1.scatter(swot_time_series.index, swot_time_series['pressure_with_DAC'] - np.mean(swot_time_series['pressure_with_DAC']), \
            label='SWOT pressure with DAC added back in', color='C0')
ax1.set_ylabel('Pressure anomaly (dbar)', color='C0') 
ax1.tick_params(axis='y', labelcolor='C0') 

ax2 = ax1.twinx()  
ax2.plot(dac['Datetime'], dac['ssh_correction_m'], label='DAC', color='C1')
ax2.set_ylabel('SSH (m)', color='C1')  
ax2.tick_params(axis='y', labelcolor='C1') 

handles, labels = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()

combined_handles = handles + handles2
combined_labels = labels + labels2

ax1.legend(combined_handles, combined_labels, loc='best')

ax1.grid(True)
plt.savefig('endeavor_BPR_SWOT_DAC.png', dpi=300, bbox_inches='tight', transparency = False)

# Remove Aviso trend from SWOT+DAC

In [ ]:
swot_time_series['detrended_pressure_with_dac'] = swot_time_series['pressure_with_DAC'] - swot_trend


fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(detided_bpr_df.index, detided_bpr_df['pressure_anomaly'], label='BPR Data', color='C0')
ax.scatter(aviso['Date'], aviso['pressure_anomaly'], label='Bottom Pressure from Aviso SSH', color='C2')
#ax.scatter(swot_time_series.index, swot_time_series['pressure_anomaly'], label='Bottom pressure from SWOT SSH', color='C4')
ax.plot(date_range, trend_values - np.mean(trend_values)-0.012, label='Aviso trend', color='C3', linestyle='--')
ax.scatter(swot_time_series.index, swot_time_series['detrended_pressure_with_dac']-np.mean(swot_time_series['detrended_pressure_with_dac']),\
           label='SWOT + DAC - Aviso trend', color='C1')

plt.legend(loc = 'upper right')
plt.ylabel('Pressure Anomaly (dbar)')
plt.grid(True)

plt.savefig('SWOT+DAC_detrended_with_aviso.png', dpi=300, bbox_inches='tight', transparent=False)

# CTD data

In [ ]:
# Original downloaded file has a frequency of 1/Hz

def read_and_preprocess(file_path):
    rows_to_skip = list(range(52)) + [54]
    df = pd.read_csv(file_path, delimiter=',', skiprows=rows_to_skip, low_memory=False)
    df = df[1:].reset_index()
    df.columns = df.columns.str.strip().str.replace('"', '')
    df = df.rename(columns={df.columns[1]: 'Time_UTC'})
    df['Time_UTC'] = pd.to_datetime(df['Time_UTC'])
    df = df.set_index('Time_UTC').resample('H').mean().reset_index()

    return df

file_pattern = 'CTD/*.csv'
files = sorted(glob(file_pattern))

dfs = []

for file in files:
    df = read_and_preprocess(file)
    dfs.append(df)

ctd = pd.concat(dfs, ignore_index=True)
ctd.drop(ctd.columns[1:10], axis=1, inplace=True)
ctd.dropna(inplace=True)
ctd = ctd.reset_index()

# Calculate pressure at the CTD from salinity, and temperature using gsw, compare to their measurement of density and pressure

In [ ]:
# These values come from here: https://data.oceannetworks.ca/DataPreview?TREETYPE=1&LOCATION=1310&DEVICECATEGORY=5&TIMECONFIG=2
ctd_depth = 2320 # m
ctd_lon = 129.0354
ctd_lat = 47.9584

def calc_sfp_steric(temp, salt, depth, lon, lat, g=9.81):
    numTimeSteps = temp.shape[0]
    pressure_gsw = np.zeros(numTimeSteps)
    density_gsw = np.zeros(numTimeSteps)
    ref_lon = lon
    ref_lat = lat

    for time_step in range(numTimeSteps):
        SP = salt[time_step]
        SA = gsw.SA_from_SP(SP, depth, ref_lon, ref_lat)
        T = temp[time_step]
        CT = gsw.CT_from_t(SA, T, depth)
        density = gsw.rho(SA, CT, depth)
        pressure = density * g * depth / 10000  # Convert to dbar
        pressure_gsw[time_step] = pressure
        density_gsw[time_step] = density

    return np.array(pressure_gsw), np.array(density_gsw)

pressure_gsw, density_gsw = calc_sfp_steric(ctd['Temperature (C)'], ctd['Practical Salinity (psu)'], \
                                            ctd_depth, ctd_lon, ctd_lat)

ctd['pressure_gsw'] = pressure_gsw
ctd['density_gsw'] = density_gsw

plt.figure(figsize=(12, 6))
plt.plot(ctd['Time_UTC'], ctd['Density (kg/m3)'] - np.mean(ctd['Density (kg/m3)']), label='ONC CTD calculation')
plt.plot(ctd['Time_UTC'], ctd['density_gsw'] - np.mean(ctd['density_gsw']), label='GSW calculation')
plt.ylabel('Density anomaly (kg/m3)')
plt.legend();

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(detided_bpr_df.index, detided_bpr_df['pressure_anomaly'], label='BPR Data', color='C0')
#ax.scatter(aviso['Date'], aviso['pressure_anomaly'], label='Bottom Pressure from Aviso SSH', color='C2')
#ax.scatter(swot_time_series.index, swot_time_series['detrended_pressure_anomaly'], label='SWOT Pressure Detrended Using Aviso', color='C1')
#ax.scatter(swot_time_series.index, swot_time_series['pressure_anomaly'], label='Bottom pressure from SWOT SSH', color='C4')

#ax.plot(date_range, trend_values - np.mean(trend_values)-0.012, label='Trend from Aviso', color='C3', linestyle='--')
ax.plot(ctd['Time_UTC'], ctd['pressure_gsw'] - np.mean(ctd['pressure_gsw']), label='Steric anomaly from CTD', color='C1')
plt.legend(loc = 'upper left')
plt.ylabel('Pressure Anomaly (dbar)')
plt.grid(True)

plt.savefig('BPR_vs_CTD_steric.png', dpi=300, bbox_inches='tight', transparent=False)

# Compare CTD and BPR pressure measurements

In [ ]:
# Detide the CTD pressure measurement



# Sampling frequency and Nyquist frequency in hours^-1
sampling_rate_hours = 1  
nyquist_frequency = 0.5 * sampling_rate_hours

# Define the cutoff frequency for 48 hours
cutoff_frequency_hours = 1 / 48

# Normalize the cutoff frequency with respect to the Nyquist frequency
normalized_cutoff_frequency = cutoff_frequency_hours / nyquist_frequency

# butterworth low pass filter
N = 6
b, a = butter(N, normalized_cutoff_frequency, btype='low', analog=False)
ctd['detided_pressure'] = filtfilt(b, a, ctd['Pressure (decibar)'])

# The 50 first rows removed due to filter initialization effects.
detided_ctd = ctd.iloc[40:]

plt.figure(figsize=(10, 4))
plt.plot(detided_bpr_df.index, detided_bpr_df['detided_bottom_pressure']- np.mean(detided_bpr_df['detided_bottom_pressure']), color='C0', label='BPR')
plt.plot(detided_ctd['Time_UTC'], detided_ctd['detided_pressure'] - np.mean(detided_ctd['detided_pressure']), color='C1', label = 'CTD Pressure')

plt.ylabel('Pressure anomaly (dbar)')
plt.legend()
plt.grid(True);
plt.savefig('BPR_vs_CTD.png', dpi=300, bbox_inches='tight')